In [157]:
import pandas as pd
import numpy as np

In [158]:
svv_credentials = open('svv_creds.json.txt').read()
import json 
svv_creds = json.loads(svv_credentials)

In [159]:
import psycopg2

def get_svv_connection():
    con = psycopg2.connect(dbname = 'svv', host = 'svv-rs-prod-bi.cjddijbnvfpr.eu-west-1.redshift.amazonaws.com', \
                           port = 5439, user = svv_creds['user'], password = svv_creds['password'])
    return con

In [160]:
query_data = "SELECT * FROM sandbox.rw_2_programme_streams_80_sep_19"

df_streams = pd.DataFrame()

try: 
    con = get_svv_connection()

    cur = con.cursor()
    cur.execute(query_data)

    colnames = [desc[0] for desc in cur.description]
    df_streams = pd.DataFrame(cur.fetchall(), columns = colnames)
    
except Exception as inst:
    print(inst)
finally:
    cur.close()
    con.close()

In [161]:
df_streams.head()

,visit_id,programme_title,min_completion,min_rank,new_rank
0,00009307-142a-53b9-9b05-22bb44e12309-156944984...,Family Guy,0.996769,2,2
1,00009c54-f00d-4c17-8042-2b5719205f36-156789396...,Yorkshire Airport,1.000064,2,2
2,00020186-8886-4f9d-8e87-93462b4a0af3-156762464...,Emmerdale,0.984438,1,1
3,00023b7f-a18d-4bf0-a25e-055bab4c070c-156924960...,A Confession,0.988696,2,2
4,0002ab14-c26a-4f8f-bdd2-cfd4bad68bcb-156953107...,Inside Prison: Britain Behind Bars,0.958904,2,2


In [162]:
df_streams = df_streams.drop("min_rank", axis = 1)

In [163]:
df_programme_1 = df_streams[df_streams.new_rank == 1]
df_programme_1.head()

,visit_id,programme_title,min_completion,new_rank
2,00020186-8886-4f9d-8e87-93462b4a0af3-156762464...,Emmerdale,0.984438,1
5,000302e1-8a3d-424d-b237-ff64fe364fa2-156845325...,A Confession,1.000029,1
6,00033c81-fbdd-4d70-b93e-d2f44c4f511f-156746301...,Coronation Street,0.980090,1
10,00064c90-84d9-58b6-81a9-8c28d3973660-156766584...,Singletown,0.992311,1
11,0006b078-1e7e-4abd-a9be-dfe5b5befee1-156841420...,Emmerdale,1.000074,1


In [164]:
df_programme_1['visit_id'].nunique()

1742019

In [165]:
df_programme_2 = df_streams[df_streams.new_rank == 2]
df_programme_2.head()

,visit_id,programme_title,min_completion,new_rank
0,00009307-142a-53b9-9b05-22bb44e12309-156944984...,Family Guy,0.996769,2
1,00009c54-f00d-4c17-8042-2b5719205f36-156789396...,Yorkshire Airport,1.000064,2
3,00023b7f-a18d-4bf0-a25e-055bab4c070c-156924960...,A Confession,0.988696,2
4,0002ab14-c26a-4f8f-bdd2-cfd4bad68bcb-156953107...,Inside Prison: Britain Behind Bars,0.958904,2
7,0003b5bd-62aa-5184-90d2-0cc6429bea81-156949509...,Coronation Street,0.977629,2


In [166]:
df_programme_2['visit_id'].nunique()

1585759

In [167]:
df_merged = df_programme_1.merge(df_programme_2, on = 'visit_id', how = 'inner')
df_merged.head()

,visit_id,programme_title_x,min_completion_x,new_rank_x,programme_title_y,min_completion_y,new_rank_y
0,00020186-8886-4f9d-8e87-93462b4a0af3-156762464...,Emmerdale,0.984438,1,Coronation Street,1.000032,2
1,00064c90-84d9-58b6-81a9-8c28d3973660-156766584...,Singletown,0.992311,1,A Confession,0.987969,2
2,0006b078-1e7e-4abd-a9be-dfe5b5befee1-156841420...,Emmerdale,1.000074,1,Coronation Street,0.988871,2
3,000bba74-aac0-47f1-932c-399b345c39fd-156805207...,The Only Way is Essex,0.995671,1,A Confession,1.000345,2
4,000bbd43-ade5-47c6-8b49-66d3dff2ca2a-156939515...,Home Fires,1.000046,1,Scott & Bailey,1.000078,2


In [168]:
df_pivot = df_merged[['programme_title_x','programme_title_y', 'visit_id']].groupby(['programme_title_x','programme_title_y']).count()

In [169]:
df_pivot.head()

visit_id
programme_title_x programme_title_y                 
5 Gold Rings      A Confession                    76
                  Absolutely Ascot                 5
                  Agatha Christie's Poirot         1
                  All Elite Wrestling              1
                  American Dad!                   10

In [170]:
df_pivot = df_pivot.reset_index()

In [171]:
df_pivot.head()

,programme_title_x,programme_title_y,visit_id
0,5 Gold Rings,A Confession,76
1,5 Gold Rings,Absolutely Ascot,5
2,5 Gold Rings,Agatha Christie's Poirot,1
3,5 Gold Rings,All Elite Wrestling,1
4,5 Gold Rings,American Dad!,10


In [172]:
df_matrix = df_pivot.pivot_table(index = 'programme_title_x', columns = 'programme_title_y', fill_value = 0)
df_matrix

visit_id               \
programme_title_y                                  5 Gold Rings A Confession   
programme_title_x                                                              
5 Gold Rings                                                  0           76   
A Confession                                                 66            0   
A Touch of Frost                                              1           22   
Absolutely Ascot                                              3          175   
Agatha Christie's Marple                                      2           36   
Agatha Christie's Poirot                                      0           59   
Ainsley's Caribbean Kitchen                                   1           47   
Alexander Armstrong in the Land of the Midnight...            0            1   
All Elite Wrestling                                           0            0   
American Dad!                                                 7          115   
Animanimals                                                   0            0   
BRDC Formula 3 Championship Highlights                        0            0   
BabyRiki                                                      0            3   
Bachelor In Paradise                                          3           18   
Banks of England                                              0            0   
Barging Round Britain with John Sergeant                      0            3   
Be Beautiful                                                  0            5   
Be Tasty                                                      0            1   
Bear Grylls Survival School                                   1            6   
Ben 10                                                        0            0   
Benidorm                                                      0           19   
Benidorm: Ten Years on Holiday                                0            7   
Billy Connolly's Great American Trail                         3          152   
Birds of a Feather                                            2            0   
Black Panther: Trouble in Wakanda                             0            0   
Bradley & Barney: Breaking Dad                                1            4   
Bridezillas                                                   2           15   
Britain's Got Talent: The Champions                         241         1313   
British Superbikes                                            0           18   
British Touring Car Championship                              0            2   
...                                                         ...          ...   
The Royal                                                     2           10   
The Rubbish World of Dave Spud                                0            6   
The Saint                                                     0            4   
The Sara Cox Show                                             1            1   
The Shawshank Redemption                                      1            9   
The Sports Stars Fishing Championship                         0            2   
The Sweeney                                                   1            4   
The Widower                                                   0           14   
This Morning                                                  8          101   
Through the Keyhole                                           1            1   
Tipping Point                                                16          122   
Tipping Point: Lucky Stars                                   85           25   
Tonight                                                       1           23   
Tonight at the London Palladium                               1            0   
Toonmarty                                                     0            0   
Totally Bonkers Guinness World Records                        0            3   
Totally Unbelievable                                   

In [173]:
df_matrix.to_csv('Sep 19 2 Programme Streams Matrix.csv')

In [183]:
df_matrix.iloc[0][:5]

          programme_title_y       
visit_id  5 Gold Rings                 0
          A Confession                76
          A Touch of Frost             0
          Absolutely Ascot             5
          Agatha Christie's Marple     0
Name: 5 Gold Rings, dtype: int64

In [185]:
df_matrix.idxmax(axis = 1)

programme_title_x
5 Gold Rings                                                  (visit_id, Who Wants to Be a Millionaire?)
A Confession                                                               (visit_id, Coronation Street)
A Touch of Frost                                                            (visit_id, Midsomer Murders)
Absolutely Ascot                                                       (visit_id, The Only Way is Essex)
Agatha Christie's Marple                                            (visit_id, Agatha Christie's Poirot)
Agatha Christie's Poirot                                                    (visit_id, Midsomer Murders)
Ainsley's Caribbean Kitchen                                                     (visit_id, A Confession)
Alexander Armstrong in the Land of the Midnight Sun      (visit_id, Heathrow: Britain's Busiest Airport)
All Elite Wrestling                                                          (visit_id, Rugby World Cup)
American Dad!                        